In [ ]:
'TensorFlow Lite Model Maker库'
'简化了在设备上ML应用程序中部署TensorFlow模型并将其转换为特定输入数据的过程。'
import numpy as np
import os

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import configs
from tflite_model_maker import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker import TextClassifierDataLoader

# 斯坦福情绪树库
# 包含用于培训的67,349条电影评论和用于验证的872条电影评论。数据集有两类：正面和负面电影评论。
data_dir = tf.keras.utils.get_file(
    fname='SST-2.zip',
    origin='https://dl.fbaipublicfiles.com/glue/data/SST-2.zip',
    extract=True
)
data_dir = os.path.join(os.path.dirname(data_dir), 'SST-2')

In [ ]:
# TensorFlow Lite Model Maker currently supports :
# MobileBERT, averaging word embeddings and BERT-Base models.

# 采用较小的模型：mobileBERT
spec = model_spec.get('mobilebert_classifier')
spec_bert = model_spec.get('bert_classifier')

# 加载特定于设备上ML应用程序的训练和测试数据，并根据特定的数据进行预处理
train_data = TextClassifierDataLoader.from_csv(
      filename=os.path.join(os.path.join(data_dir, 'train.tsv')),
      text_column='sentence',
      label_column='label',
      model_spec=spec,
      delimiter='\t',
      is_training=True)
test_data = TextClassifierDataLoader.from_csv(
      filename=os.path.join(os.path.join(data_dir, 'dev.tsv')),
      text_column='sentence',
      label_column='label',
      model_spec=spec,
      delimiter='\t',
      is_training=False)
# 自定义TensorFlow模型
model = text_classifier.create(train_data, model_spec=spec,epochs=1)
model.summary()

In [ ]:
# 评估模型
loss, acc = model.evaluate(test_data)
# 量化模型，导出为带有元数据的TensorFlow Lite模型。
config = configs.QuantizationConfig.create_dynamic_range_quantization(
    optimizations=[tf.lite.Optimize.OPTIMIZE_FOR_LATENCY]
)
config.experimental_new_quantizer = True
# 使用元数据将现有模型转换为TensorFlow Lite模型格式，以后可在设备上的ML应用程序中使用。
# 标签文件和vocab文件嵌入在元数据中。TFLite的默认文件名是model.tflite。
model.export(export_dir='mobilebert/', quantization_config=config)
# 使用evaluate_tflite方法评估tflite模型以获取其准确性。
accuracy = model.evaluate_tflite('average_word_vec/model.tflite', test_data)